In [ ]:
import json
import os
import requests
from urllib.parse import urlparse

In [ ]:
# Directory to save downloaded MP3 files
output_directory = 'data/downloaded_mp3s'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

In [ ]:
with open("har_files/har_data.har", "r", encoding="utf-8") as file:
    input_data = json.load(file)

input_data

mp3_urls = []

entries = input_data["log"]["entries"]

for i, entry in enumerate(entries):
    request = entry["request"]
    url = request["url"]
    if url.endswith(".mp3"):
        mp3_urls.append(url)

mp3_urls

In [ ]:
# Custom headers to be added to the requests
custom_headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'identity;q=1, *;q=0',
    'Accept-Language': 'en-US,en;q=0.9',
    'Dnt': '1',
    'Host': 'learn101.org',
    'Proxy-Connection': 'keep-alive',
    'Range': 'bytes=0-',
    'Referer': 'http://learn101.org/sound/swahili/swahili_one.mp3',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

# Download MP3 files from the URLs
# mp3_url = mp3_urls[0]
for mp3_url in mp3_urls:
    try:
        custom_headers["Referer"] = mp3_url
        response = requests.get(mp3_url, headers=custom_headers, stream=True)
        if response.status_code == 200 or response.status_code == 206:
            # Extract the filename from the URL
            url_path = urlparse(mp3_url).path
            file_name = os.path.join(output_directory, os.path.basename(url_path))

            # Save the MP3 file locally with the same name as in the URL
            with open(file_name, 'wb') as mp3_file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        mp3_file.write(chunk)
            print(f'Downloaded: {file_name}')
        else:
            print(f'Failed to download: {mp3_url}, Status Code: {response.status_code}')
    except Exception as e:
        print(f'Error downloading: {mp3_url}, Error: {str(e)}')

print('Download complete.')
